# The Baseline RAG Pipeline

---

This notebook is your **starting point**. The pipeline is already built and working: run it, explore its outputs, question it, and find its limits.

**How to use this notebook**

| | |
|---|---|
| 📖 **Read** | The explanations use plain language, no coding background needed |
| ▶️ **Run** | Execute cells top to bottom with Shift+Enter to see the pipeline in action |
| 💬 **Discuss** | Talk about the outputs with your peers, do they make sense? Would you trust them? |
| 🔧 **Experiment** | Modify queries, tweak parameters, break things on purpose |
| 🚀 **Extend** | The Tasks section points to what you can take further |

> You don't need to understand every line of code. Focus on what the system gets right and wrong and on thinking about how this would apply in your own context.

---

## What This Notebook Covers

**Retrieval-Augmented Generation (RAG)** combines a search engine with an AI assistant. Instead of the AI making things up from memory, it first searches your documents and then answers based on what it finds. The answer can always be traced back to a source.

```
Your question  ->  Search your documents  ->  AI answers using only those documents
```

| Notebook | Focus |
|---|---|
| **Baseline (this notebook)** | Working baseline prototype |
| Feature Track 1 | How documents are split into searchable pieces |
| Feature Track 2 | How to measure answer quality |
| Feature Track 3 | Reliable, structured outputs |
| Feature Track 4 | Better retrieval strategies |
| Feature Track 5 | Multi-step agent workflows |
| Feature Track 6 | Connecting to the chat frontend |

---

## Why RAG? The Problem with a Standalone LLM

### The Scenario
**PrimePack AG** buys packaging materials (pallets, cardboard boxes, tape) from multiple suppliers. Sustainability claims are increasingly scrutinised by customers and regulators. Employees need to answer questions like:
> *"What is the GWP of the Logypal 1 pallet, and is the figure verified?"*  
> *"Can we tell a customer that the tesa tape is PFAS-free?"*  
> *"Which of our suppliers have a certified EPD?"*

### Why Not Just Ask ChatGPT?
A general-purpose LLM has three fundamental problems for this task:

| Problem | Why It Matters |
|---|---|
| **Internal document** | LLMs don't know about internal company documents. |
| **Hallucination** | When asked about unknown products the LLM invents plausible-sounding but false figures. |
| **No evidence trail** | Even when correct, a raw LLM answer cannot be traced back to a source document. |

### The RAG Solution
RAG adds a **retrieval step** between the user's question and the LLM:

```
 Documents ──► Chunker ──► Embedder ──► Vector DB
                                              │
 User query ─────────────────► Embedder ─────►  Retriever ──► Top-k Chunks
                                                                      │
                                                               LLM + Prompt
                                                                      │
                                                               Answer + Sources
```

The LLM only sees documents that are **actually in the corpus**. The answer can be traced to specific source chunks. If the corpus does not contain the answer, the LLM is instructed to say so.

### What RAG Does *Not* Fix
RAG shifts the problem from hallucination to **retrieval quality**. If the right chunk is not retrieved, the answer will still be wrong (or absent). The later feature tracks address exactly this: better chunking, better retrieval, and better output structure.

---

## Core Concepts

### Chunks

A **chunk** is a short excerpt from a source document, a section of a PDF, one sheet of a spreadsheet, or one heading-delimited paragraph of a Markdown file. Chunks are the unit of indexing and retrieval.

```python
@dataclass
class Chunk:
    id: str           # unique identifier
    title: str        # e.g. section heading
    content: str      # the text that gets embedded
    metadata: dict    # source_file, page, ...
```

### Embeddings
An **embedding** converts text to a dense numeric vector (e.g. 384 dimensions). Semantically similar texts produce similar vectors. Here we use `all-MiniLM-L6-v2`, a compact local model that runs without an API key.

### Vector Store (ChromaDB)
A **vector store** persists chunk embeddings on disk and supports approximate nearest-neighbour search. Given a query embedding, it returns the `top_k` most similar chunks in milliseconds.

### Retriever
A **retriever** wraps a vector store and exposes a single `retrieve(query)` method. The baseline uses a `VectorStoreRetriever` with `top_k=5`.

### RAG Agent
The **RAG agent** combines a retriever and an LLM. Its `answer()` method:
1. Embeds the query
2. Retrieves the top-k chunks
3. Formats chunks as XML `<source>` tags in the prompt
4. Calls the LLM and returns the answer + cited sources

---

## Setup

**Prerequisites:** `conversational-toolkit` and `backend` must be installed in editable mode (`pip install -e conversational-toolkit/ && pip install -e backend/`). For the **Ollama** backend, start `ollama serve` and pull the model (`ollama pull mistral-nemo:12b`). For **OpenAI**, set `OPENAI_API_KEY` in your environment.

In [ ]:
from pathlib import Path

from conversational_toolkit.agents.base import QueryWithContext
from conversational_toolkit.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from conversational_toolkit.retriever.vectorstore_retriever import VectorStoreRetriever

from sme_kt_zh_collaboration_rag.feature0_baseline_rag import (
    load_chunks,
    inspect_chunks,
    build_vector_store,
    inspect_retrieval,
    build_agent,
    build_llm,
    ask,
    DATA_DIR,
    VS_PATH,
    EMBEDDING_MODEL,
    RETRIEVER_TOP_K,
)

# Choose your LLM backend: "ollama" (local, requires `ollama serve`) or "openai" (requires OPENAI_API_KEY)
BACKEND = "ollama"  # set this before running

if not BACKEND:
    raise ValueError(
        'BACKEND is not set. Edit the line above and set it to "ollama", or "openai".\n'
        "See Renku_README.md for setup instructions."
    )

ROOT = Path().resolve().parents[1]
print(f"Project root : {ROOT}")
print(f"Data dir     : {DATA_DIR}")
print(f"Vector store : {VS_PATH}")
print(f"LLM backend  : {BACKEND}")

---

## Before the RAG Pipeline: The LLM on Its Own

A **large language model (LLM)** is a neural network trained on billions of words of text. It can summarise documents, answer questions, and generate structured output, but only from knowledge baked into its weights during training. It has no direct access to your internal documents.

Before building the RAG pipeline, let's interact with the LLM directly to understand what it can and cannot do on its own.

In [ ]:
from conversational_toolkit.llms.base import LLMMessage, Roles

# Reuse the backend you chose in the Setup cell above
llm_standalone = build_llm(backend=BACKEND)

# A question the LLM can answer from general training data
general_question = "What does GWP stand for, and what unit is it typically measured in?"

response_general = await llm_standalone.generate(
    [LLMMessage(role=Roles.USER, content=general_question)]
)
print("---------------------------")
print(f"Q: {general_question}\n")
print("---------------------------")
print(f"A: {response_general.content}")

The LLM answers that correctly, GWP is a well-known concept covered in its training data.

Now ask something specific to PrimePack AG's product portfolio:

In [ ]:
# A product-specific question the LLM has never seen in training
primepack_question = "What is the Global Warming Potential (GWP) of the Logypal 1 pallet sold by PrimePack AG, and is the figure third-party verified? Provide the link to PrimePack AG's official website."

response_pp = await llm_standalone.generate(
    [LLMMessage(role=Roles.USER, content=primepack_question)]
)
print("---------------------------")
print(f"Q: {primepack_question}\n")
print("---------------------------")
print(f"A: {response_pp.content}")

PrimePack AG is a fictional company, no training data exists for this product
- If the model gave a specific figure or website link: it is hallucinated.
- If the model said 'I don't know': that is honest, but still not useful.

Either way, the LLM cannot provide the actual figure with a verifiable source.

> **Task: Compare LLM Backends**
> 1. **Switch backends.** Change BACKEND to "ollama" (or "openai" if you started with Ollama) and re-run the two question cells above. Does one model hallucinate a specific figure or website link while the other declines? How confident does each answer sound?
> 2. **Change the question.** Replace the tesa question with something you could imagine being asked in a real supplier audit. Does the standalone LLM give you an answer you would trust?
> 3. **Note the pattern.** Regardless of whether the model hallucinates or says "I don't know", ask: could you send this response to a customer? What is missing?

### Why do different models behave differently?

**OpenAI models** (GPT-4o, GPT-4o-mini) are extensively trained with human feedback (Reinforcement Learning from Human Feedback(RLHF)) to decline when they lack reliable information. For a fictional company like PrimePack AG, with no public web presence, the model has learned to say "I don't know" rather than confabulate a specific figure.

**Smaller local models** (Mistral, LLaMA 7–13 B) are typically less safety-fine-tuned. Without the reinforcement signal that penalises confident wrong answers, they are more likely to generate a plausible-sounding but fabricated answer.

**The problem in either case:** "I don't know" and a hallucinated answer are equally useless to an employee who needs to respond to a CSRD audit. The correct response, *"The verified GWP is X kg CO₂e according to the 2023 EPD (source: EPD_pallet_relicyc_logypal1.pdf)"*, requires access to the actual document.

---

## Choosing a Backend: OpenAI API vs Local Models

Two LLM backends are available in this workshop. Both expose the same interface, switching requires changing a single variable.

### Comparison

| | **OpenAI API** (`gpt-4o-mini`, `gpt-4o`, …) | **Ollama -> local** (`mistral-nemo:12b`, `llama3.2`, …) |
|---|---|---|
| **Data security** | Queries and document chunks are sent to OpenAI's servers. You can request zero-data-retention. | Everything stays on-premise. Nothing leaves the machine. Suitable for confidential documents without any external data agreements. |
| **Model capability** | State-of-the-art. Follows complex instructions reliably, structures output well, handles edge cases. `gpt-4o-mini` is the default for this workshop, it is much cheaper than `gpt-4o` with most of the capability for RAG tasks. | Smaller models (7–13 B parameters) are weaker on complex reasoning and strict rule-following. For straightforward retrieval and summarisation tasks the quality gap narrows considerably. |
| **Cost** | Per-token billing. A typical RAG query costs a fraction of a cent. See the cost estimation section below. | No API cost: you pay for hardware (CPU/GPU) and electricity. |
| **Latency** | ~1–2 s per query (network + model time) | CPU-only: ~20–60 s. GPU: ~2–5 s |
| **Setup** | One API key, no local hardware required | `ollama serve` + model download |

### Self-Hosting Larger Models

The quality gap between a 12 B local model and GPT-4o can be substantially closed at larger model sizes:

- **LLaMA 3.1 70 B, Mistral Large 2, Qwen 2.5 72 B**: run on GPUs. Quality can approach GPT-4o on structured tasks like RAG.
- **Quantised models (GGUF / GPTQ):** Reduce memory requirements by 50–75% with a modest quality trade-off, making larger models accessible on smaller hardware.
- **Production stacks:** `vLLM` and `llama.cpp` server provide OpenAI-compatible APIs with batching and much higher throughput than `ollama` alone. 

**For this workshop** `gpt-4o-mini` (OpenAI) and `mistral-nemo:12b` (Ollama) are both sufficient to demonstrate the full RAG pipeline.

---

## Cost Estimation

API costs scale with the number of tokens processed. **Input tokens** (your system prompt and retrieved document chunks) are cheaper than **output tokens** (the model's generated answer).

OpenAI API pricing for all models can be found [here](https://developers.openai.com/api/docs/pricing/). As an example, prices for `gpt-4o-mini` are:

| Token type | Price |
|---|---|
| Input | $0.15 / 1 M tokens |
| Output | $0.60 / 1 M tokens |

A rough rule of thumb: **1 token ≈ 4 characters** of English text.

In [ ]:
# Cost estimation for gpt-4o-mini
INPUT_PRICE_PER_TOKEN = 0.15 / 1_000_000  # USD
OUTPUT_PRICE_PER_TOKEN = 0.60 / 1_000_000  # USD


def estimate_tokens(text: str) -> int:
    """Rough estimate: 1 token ≈ 4 characters."""
    return max(1, len(text) // 4)


def estimate_cost(input_text: str, output_text: str) -> dict:
    input_tok = estimate_tokens(input_text)
    output_tok = estimate_tokens(output_text)
    cost = input_tok * INPUT_PRICE_PER_TOKEN + output_tok * OUTPUT_PRICE_PER_TOKEN
    return {"input_tokens": input_tok, "output_tokens": output_tok, "cost_usd": cost}


# Simulate a typical RAG query: system prompt + 5 retrieved chunks + user question --> short generated answer
example_input = (
    "You are a helpful AI assistant specialised in sustainability for PrimePack AG. "
    "Answer only using the provided document excerpts. Cite your sources.\n\n"
    "Source: EPD_pallet_relicyc_logypal1.pdf\n"
    "The Logypal 1 pallet has a declared GWP of 3.2 kg CO\u2082e per functional unit (A1\u2013A3), "
    "verified by an independent third-party auditor under ISO\u202014044.\n\n"
    "Source: ART_product_catalog.md\n"
    "The Logypal 1 (Product ID: 20-100) is a recycled-plastic pallet supplied by Relicyc. "
    "It is listed as the primary pallet for heavy-duty use in the PrimePack AG portfolio.\n\n"
    "[... 3 more retrieved chunks ...]\n\n"
    "Q: What is the GWP of the Logypal 1 pallet, and is it verified?"
)
example_output = (
    "The Logypal 1 pallet has a GWP of 3.2\u202fkg\u202fCO\u2082e per functional unit (A1\u2013A3), "
    "according to the third-party verified EPD (EPD_pallet_relicyc_logypal1.pdf). "
    "The figure has been independently audited under ISO\u202014044."
)

info = estimate_cost(example_input, example_output)
print(f"Input  : ~{info['input_tokens']:>5,} tokens (prompt + chunks + question)")
print(f"Output : ~{info['output_tokens']:>5,} tokens (generated answer)")
print(f"Cost   : ${info['cost_usd']:.6f} per query")
print()
print(f"At 1,000 queries / day     ->  ${info['cost_usd'] * 1_000:>8.4f} / day")
print(f"At 10,000 queries / day    ->  ${info['cost_usd'] * 10_000:>8.4f} / day")
print(f"At 1,000,000 queries / day ->  ${info['cost_usd'] * 1_000_000:>8.2f} / day")

The largest cost driver is context length. More retrieved chunks = more input tokens.
top_k=5 (~2,000 input tokens) is a reasonable starting point for RAG.

> **Consider:** For your use case, how many queries would the system handle per day? At what volume does the per-query cost become meaningful? Would data-security requirements push you towards a local model even at lower throughput?

---
# RAG Pipeline

Now that we have seen what an LLM can and cannot do on its own, we are ready to build the retrieval layer that makes it genuinely useful. The following five steps walk through the full RAG pipeline end-to-end, from loading the documents all the way to a sourced answer.

```
 Documents ──► Chunker ──► Embedder ──► Vector DB
                                              │
 User query ─────────────────► Embedder ─────►  Retriever ──► Top-k Chunks
                                                                      │
                                                               LLM + Prompt
                                                                      │
                                                               Answer + Sources
```

---
## Step 1: Load and Chunk Documents

The `load_chunks()` function walks `data/` and dispatches each file to the right chunker:

| Extension | Chunker | Strategy |
|---|---|---|
| `.pdf` | `PDFChunker` | Convert to Markdown via `pymupdf4llm`, split on `#` headings |
| `.xlsx`, `.xls` | `ExcelChunker` | One chunk per sheet, serialised as a Markdown table |
| `.md`, `.txt` | `MarkdownChunker` | Split on `#` headings |

The result is a flat `list[Chunk]`, the same structure regardless of the original format.

You can use `max_files=5` here for speed. Remove the limit (or set `None`) to load the full corpus.

> **Feature Track 1** explores the importance of ingestion and alternative chunking strategies in more depth.

In [ ]:
# Load documents from DATA_DIR and split them into chunks.
chunks = load_chunks(max_files=None)
# Print a statistical summary and sampled content for visual inspection.
inspect_chunks(chunks)

# Print size distribution
char_lengths = [len(c.content) for c in chunks]
over_limit = sum(1 for n in char_lengths if n > 1024)
print(f"\nChunks total       : {len(chunks)}")
print(f"Mean length (chars): {sum(char_lengths) // len(char_lengths)}")
print(f"Over 1024-char limit (≈256 tok embedding limit): {over_limit} / {len(chunks)}")
print("\nSuccessfully loaded and chunked the documents!")

### What a Chunk Looks Like

Each chunk carries a `title` (the heading), the raw text `content`, and a `metadata` dict
with the source file name. This metadata is returned alongside the answer so the user can
trace every claim back to its origin document.

In [ ]:
# Print 3 representative chunks
for c in chunks[:3]:
    print(f"--- [{c.metadata.get('source_file', '?')}] ---")
    print(f"Title  : {c.title!r}")
    print(f"Length : {len(c.content)} chars")
    print(f"Preview: {c.content[:200].strip()!r}")
    print()

---

## Step 2: Embed Chunks and Build the Vector Store

`SentenceTransformerEmbeddings` converts every chunk's `content` to a 384-dimensional vector using `all-MiniLM-L6-v2`. The resulting matrix (shape `[n_chunks, 384]`) is inserted into a persistent `ChromaDBVectorStore`.

**On subsequent runs**, leave `reset=False` (the default) to skip re-embedding, it takes time and the store on disk is already correct. Pass `reset=True` only when the corpus or chunking strategy changes.

---

### Embedding Models
Two model families are implemented in the toolkit. The choice affects retrieval quality, cost, context limit, and data security.

| | `all-MiniLM-L6-v2` (default) | Other SentenceTransformer models | `text-embedding-3-small` (OpenAI) |
|---|---|---|---|
| **Dimensions** | 384 | 384–1024 (model-dependent) | 1024 (toolkit setting) |
| **Context limit** | 256 tokens | 256–8 192 (model-dependent) | 8 191 tokens |
| **Cost** | Free, local | Free, local | ~$0.02 / 1 M tokens |
| **Data security** | Fully local | Fully local | Sent to OpenAI |
| **Quality** | Good for short, focused text | Varies; some match OpenAI | State-of-the-art |
| **Setup** | No API key | No API key | `OPENAI_API_KEY` required |

**SentenceTransformer: free alternatives from HuggingFace**
Any model from HuggingFace that is compatible with the `sentence-transformers` library works with our `SentenceTransformerEmbeddings` by passing a different `model_name`. Browse quality rankings on the [MTEB leaderboard](https://huggingface.co/spaces/mteb/leaderboard).

Hardware matters: larger models are slower on CPU. A useful rule of thumb: models under ~150 MB run comfortably on CPU; larger models benefit from a GPU.

| Model | Size | CPU speed | Max input tokens (truncation limit) | Notes |
|---|---|---|---|---|
| `all-MiniLM-L6-v2` | 90 MB | Very fast | 256 | Default; good for short technical text |
| `sentence-transformers/all-mpnet-base-v2` | 420 MB | Moderate | 384 | Better English quality, same 512-token limit |
| `BAAI/bge-m3` | 2.3 GB | Very slow, GPU recommended | 8192 | Best multilingual quality; 8192-token limit |
| `thenlper/gte-large` | 1.3 GB | Slow, GPU recommended | 512 | Strong English quality |

> On Renku (CPU-only sessions), the top two rows are practical choices. 

**OpenAI embeddings**
`OpenAIEmbeddings` from `conversational_toolkit.embeddings.openai` calls the OpenAI API. The toolkit requests 1024 dimensions using OpenAI's Matryoshka dimension reduction, a technique that allows truncating full embeddings to a smaller size with minimal quality loss. Requires `OPENAI_API_KEY`.

```python
from conversational_toolkit.embeddings.openai import OpenAIEmbeddings
embedding_model = OpenAIEmbeddings(model_name="text-embedding-3-small")
```

---

### Vector Store

A vector store persists chunk embeddings on disk and provides approximate nearest-neighbour search. Two implementations are in the toolkit.

**`ChromaDBVectorStore`** (used in this notebook)
- Embedded database: no separate server process, data stored as files on disk at `VS_PATH`.
- Survives session restarts, which is why `reset=False` is safe by default.
- Uses L2 distance for search. For unit-length vectors (which both embedding models produce) this gives the same ranking as cosine similarity -> different numbers, identical top-k order.
- Well-suited for corpora up to ~100 k chunks. Not designed for concurrent writes or multi-user access.

**`PGVectorStore`** (also in the toolkit)
- PostgreSQL with the `pgvector` extension. Requires a running Postgres instance.
- Uses cosine similarity natively (also supports other).
- Supports rich metadata filtering, concurrent reads and writes, and standard SQL queries alongside vector search.
- The right choice when you already have a Postgres infrastructure, need concurrent access, or want to combine vector search with relational data.

In [32]:
embedding_model = SentenceTransformerEmbeddings(model_name=EMBEDDING_MODEL)
print(f"Embedding model: {EMBEDDING_MODEL}")

# Set reset=True to rebuild the store from scratch
vector_store = await build_vector_store(
    chunks, embedding_model, db_path=VS_PATH, reset=False
)
print("Vector store ready.")

2026-02-22 13:23:56.861 | DEBUG    | conversational_toolkit.embeddings.sentence_transformer:__init__:57 - Sentence Transformer embeddings model loaded: sentence-transformers/all-MiniLM-L6-v2 with kwargs: {}
2026-02-22 13:23:56.869 | INFO     | sme_kt_zh_collaboration_rag.feature0_baseline_rag:build_vector_store:260 - Vector store already contains 78 chunks — skipping embedding.


Embedding model: sentence-transformers/all-MiniLM-L6-v2
Vector store ready.


### Similarity in Embedding Space

Embeddings that are close in vector space share semantic meaning. The cell below embeds several sentences and measures their cosine similarity: a value between -1 (opposite) and 1 (identical). You can change the sentences to see the impact on cosine similarity.

In [33]:
import numpy as np

sentence1 = "carbon footprint of a pallet"
sentence2 = "GWP value for the Logypal 1"
sentence3 = "PFAS-free tape declaration"
sentence4 = "the annual report of a software firm"


async def cosine_similarity(a: str, b: str) -> float:
    vecs = await embedding_model.get_embeddings([a, b])
    return float(
        np.dot(vecs[0], vecs[1]) / (np.linalg.norm(vecs[0]) * np.linalg.norm(vecs[1]))
    )


pairs = [
    (sentence1, sentence2),
    (sentence1, sentence3),
    (sentence1, sentence4),
]

print("Cosine similarities:")
for a, b in pairs:
    sim = await cosine_similarity(a, b)
    print(f"{sim:.3f}  -->  {a!r}  vs  {b!r}")

2026-02-22 13:24:04.480 | DEBUG    | conversational_toolkit.embeddings.sentence_transformer:get_embeddings:76 - sentence-transformers/all-MiniLM-L6-v2 embeddings size: (2, 384)
2026-02-22 13:24:04.492 | DEBUG    | conversational_toolkit.embeddings.sentence_transformer:get_embeddings:76 - sentence-transformers/all-MiniLM-L6-v2 embeddings size: (2, 384)
2026-02-22 13:24:04.502 | DEBUG    | conversational_toolkit.embeddings.sentence_transformer:get_embeddings:76 - sentence-transformers/all-MiniLM-L6-v2 embeddings size: (2, 384)


Cosine similarities:
0.133  -->  'carbon footprint of a pallet'  vs  'GWP value for the Logypal 1'
-0.064  -->  'carbon footprint of a pallet'  vs  'PFAS-free tape declaration'
0.014  -->  'carbon footprint of a pallet'  vs  'the annual report of a software firm'


### Comparing Embedding Models on Your Documents

A quick way to compare models without running a full evaluation: pick a query, a relevant chunk, and an irrelevant chunk, then measure the **cosine similarity gap** -> how much more similar the relevant chunk is to the query than the irrelevant one. A larger gap means the model discriminates better between useful and noise results, which translates directly to higher retrieval precision.

The cell below runs this for three CPU-friendly models (and OpenAI if the key is set). The HuggingFace models are downloaded on first use (~400 MB each, takes a minute).

In [ ]:
import os
from conversational_toolkit.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from conversational_toolkit.embeddings.openai import OpenAIEmbeddings


def cosine_sim(a: np.ndarray, b: np.ndarray) -> float:
    return float(np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b)))


# Test query and two chunks from the corpus
query = "What is the carbon footprint of the Logypal 1 pallet?"

chunk_relevant = "Logypal 1 — GWP: 3.2 kg CO2e per functional unit (A1-A3). Figure verified by independent third-party auditor under ISO 14044."
chunk_irrelevant = "PrimePack AG Supplier Code of Conduct. All suppliers must comply with applicable environmental regulations and report annually on progress."

# Models to compare (CPU-friendly by default)
models_to_compare: dict = {
    "all-MiniLM-L6-v2        (90 MB, 384-dim)": SentenceTransformerEmbeddings(
        "all-MiniLM-L6-v2"
    ),
    "all-mpnet-base-v2       (420 MB, 768-dim)": SentenceTransformerEmbeddings(
        "sentence-transformers/all-mpnet-base-v2"
    ),
    "multilingual-MiniLM-L12 (470 MB, 384-dim)": SentenceTransformerEmbeddings(
        "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
    ),
}
# Add OpenAI if key is available
if os.getenv("OPENAI_API_KEY"):
    models_to_compare["text-embedding-3-small  (API, 1024-dim)"] = OpenAIEmbeddings(
        "text-embedding-3-small"
    )

# Run comparison
print("---------------------------")
print(f"Query     : {query!r}")
print(f"Relevant  : {chunk_relevant[:70]!r}...")
print(f"Irrelevant: {chunk_irrelevant[:70]!r}...")
print()
print(f"{'Model':<44}  {'sim(relevant)':>13}  {'sim(irrelevant)':>15}  {'gap':>6}")
print("-" * 84)

for name, model in models_to_compare.items():
    vecs = await model.get_embeddings([query, chunk_relevant, chunk_irrelevant])
    sim_rel = cosine_sim(vecs[0], vecs[1])
    sim_irr = cosine_sim(vecs[0], vecs[2])
    print(f"{name:<44}  {sim_rel:>13.3f}  {sim_irr:>15.3f}  {sim_rel - sim_irr:>6.3f}")

print("\nGap = sim(relevant) - sim(irrelevant). Larger gap -> better discrimination.")

2026-02-22 13:28:04.896 | DEBUG    | conversational_toolkit.embeddings.sentence_transformer:__init__:57 - Sentence Transformer embeddings model loaded: all-MiniLM-L6-v2 with kwargs: {}
2026-02-22 13:28:07.350 | DEBUG    | conversational_toolkit.embeddings.sentence_transformer:__init__:57 - Sentence Transformer embeddings model loaded: sentence-transformers/all-mpnet-base-v2 with kwargs: {}
2026-02-22 13:28:10.111 | DEBUG    | conversational_toolkit.embeddings.sentence_transformer:__init__:57 - Sentence Transformer embeddings model loaded: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2 with kwargs: {}
2026-02-22 13:28:10.141 | DEBUG    | conversational_toolkit.embeddings.openai:__init__:20 - OpenAI embeddings model loaded: text-embedding-3-small
2026-02-22 13:28:10.278 | DEBUG    | conversational_toolkit.embeddings.sentence_transformer:get_embeddings:76 - all-MiniLM-L6-v2 embeddings size: (3, 384)
2026-02-22 13:28:10.324 | DEBUG    | conversational_toolkit.embeddings.senten

Query     : 'What is the carbon footprint of the Logypal 1 pallet?'
Relevant  : 'Logypal 1 — GWP: 3.2 kg CO2e per functional unit (A1-A3). Figure verif'...
Irrelevant: 'PrimePack AG Supplier Code of Conduct. All suppliers must comply with '...

Model                                         sim(relevant)  sim(irrelevant)     gap
------------------------------------------------------------------------------------
all-MiniLM-L6-v2        (90 MB, 384-dim)              0.573            0.065   0.509
all-mpnet-base-v2       (420 MB, 768-dim)             0.582            0.287   0.295
multilingual-MiniLM-L12 (470 MB, 384-dim)             0.675            0.166   0.509


2026-02-22 13:28:10.521 | INFO     | conversational_toolkit.embeddings.openai:get_embeddings:38 - OpenAI embeddings shape: (3, 1024)


text-embedding-3-small  (API, 1024-dim)               0.597            0.295   0.302

Gap = sim(relevant) - sim(irrelevant). Larger gap -> better discrimination.
This is a quick proxy for retrieval quality. Feature Track 2 runs a proper evaluation.


**Disclaimer**: This is a quick example, not a rigorous evaluation. A single (query, chunk) pair can be misleading — one model may score higher here and worse on a different example. Reliable model selection requires testing across many diverse queries and aggregating a metric like MRR or NDCG. Feature Track 2 shows how to do this systematically.

> Task: Probe the comparison with your own examples
> 1. Replace query, chunk_relevant, and chunk_irrelevant with other combinations 
> 2. Try a query where your phrasing differs (e.g. "carbon emissions" instead of "GWP"). Does any model bridge the gap better?
> 3. Try a query in German. Does paraphrase-multilingual-MiniLM-L12-v2 show a larger gap than the English-only default?

---

## Step 3: Inspect Retrieval (Before the LLM Sees Anything)

This is the **most important diagnostic step** in the whole pipeline:

> If the retrieved chunks are wrong, the final answer will be wrong regardless of how good the LLM is.

`inspect_retrieval()` runs the query through the embedding model, fetches the top-k most similar chunks from ChromaDB, and prints them with scores. Use this to verify that relevant documents are in the index, tune `top_k`, compare different query phrasings, and identify retrieval gaps before blaming the LLM.

The **similarity score** is the L2 distance, range [0,4], lower = more similar. L2 distance is used becuase it works for any vectors, normalised or not. Cosine similarity only makes sense for direction (magnitude doesn't matter), so it requires that vectors be unit-length to be meaningful. L2 makes no such assumption, making it the safer general default. ChromaDB defaults to L2 because it's simpler to compute and works even if vector magnitudes vary. Since our embedding model always produces equal-length vectors, we get cosine-equivalent ranking. The score numbers look different, but the top-5 results would be identical either way.

In [ ]:
QUERY = "What materials is the Logypal 1 pallet made from?"

results = await inspect_retrieval(QUERY, vector_store, embedding_model)

### Retrieval for a Product Outside the Portfolio

The PrimePack AG product catalog defines the portfolio boundary. The **Lara Pallet** is not in the catalog, it does not exist. Watch which chunks are returned and what scores they have. A **higher** minimum score (large L2 distance) signals *weaker semantic match*.

In [ ]:
QUERY_OOK = "What materials is the Lara pallet made from?"

results_ook = await inspect_retrieval(QUERY_OOK, vector_store, embedding_model)

> **Observation:** The retriever always returns the *closest* chunks it can find, it has no concept of "no match". For an unknown product the L2 distances are **higher** (the closest chunks are still about other pallets), but without a score-threshold guard the LLM receives those chunks anyway and may silently answer about the wrong product.
> **Feature Track 3** shows how to combat this issue.

---

## Step 4: Build the RAG Agent

`build_agent()` assembles the three components:

```
VectorStoreRetriever
    └─ ChromaDBVectorStore (on disk, persists across runs)
    └─ SentenceTransformerEmbeddings

RAG Agent
    ├─ LLM (Ollama / OpenAI)
    ├─ Retriever
    └─ System prompt
```

### The System Prompt

The system prompt is the primary lever for controlling LLM behaviour. It is prepended to every conversation and defines the rules the model must follow:

```
You are a helpful AI assistant specialised in sustainability and product compliance
for PrimePack AG.

You will receive document excerpts relevant to the user's question.
Produce the best possible answer using only the information in those excerpts.

Rules:
- Use the provided excerpts as your only source of truth. Do not rely on outside knowledge.
- Use all relevant excerpts when forming your answer.
- If the answer cannot be found in the excerpts, clearly say that you do not know.
- Always cite the source document for any claim you make.
- If excerpts contain conflicting information, report both values and flag the conflict.
- Distinguish between third-party verified claims (EPDs) and self-declared supplier claims.
```

Each rule addresses a concrete failure mode:

| Rule | Failure mode it prevents |
|---|---|
| Only use provided excerpts | Hallucination from general knowledge |
| Say "I don't know" if not found | Confident wrong answer about missing products |
| Cite the source document | Untraceable claims |
| Flag conflicting information | Silently picking the wrong figure when sources disagree |
| Distinguish EPD vs. self-declared | Presenting unverified marketing claims as verified facts |

> **Note:** Stronger models (especially GPT-4o) follow these rules reliably even with a minimal prompt. Smaller local models (Ollama `mistral-nemo:12b`) benefit significantly from explicit, detailed rules. Section 4 probes these failure modes — if you are using OpenAI you may need to use a weaker model or a more leading query to observe them.

In [ ]:
llm = build_llm(backend=BACKEND)
agent = build_agent(
    vector_store=vector_store,
    embedding_model=embedding_model,
    llm=llm,
    top_k=RETRIEVER_TOP_K,
    number_query_expansion=0,  # 0 = no expansion; see Feature Track 4 for more
)
print("RAG agent assembled.")

---

## Step 5: Ask a Question

`ask()` sends the query to the agent and returns the answer string. The internal flow is:

1. Embed the query
2. Retrieve top-k chunks
3. Build the prompt: `<system>` + `<sources>` XML block + user question
4. Generate the answer with the LLM
5. Return the answer and a list of cited source chunks

In [ ]:
QUERY = "What materials is the Logypal 1 pallet made from?"

answer = await ask(agent, QUERY)

---

## Step 5b: Query Expansion

A single query embedding is a single point in vector space. If the user's phrasing differs from the document's phrasing, the best matching chunk may not rank in the top-k.

**Query expansion** generates several alternative formulations of the same question, retrieves independently for each, and merges the results. The agent already supports this via the `number_query_expansion` parameter.

```
Original query: "What is the carbon footprint of the Logypal 1?"
        │
        ▼  LLM generates N variants
        ├── "GWP of the Logypal 1 pallet in kg CO₂e"
        ├── "Environmental impact, global warming potential, Logypal 1"
        └── "Relicyc pallet carbon emissions A1-A3"
        │
        ▼  Retrieve top-k for each variant → deduplicate → merge
        └── Combined result set → LLM generates final answer
```

The cost is one extra LLM call per expansion (to generate the variants), but retrieval recall typically improves substantially for domain-specific vocabulary differences.

> **Feature Track 4** explores this and other advanced retrieval strategies (hybrid search, re-ranking) in depth.

In [ ]:
# Compare retrieval with and without query expansion

QUERY_EXPANSION = "What is the carbon footprint of the Logypal 1?"

# Without expansion — single query, top-5 chunks
results_no_exp = await inspect_retrieval(QUERY_EXPANSION, vector_store, embedding_model)

print("\n--- With 3 expanded queries ---\n")

# With expansion — 3 variants generated, deduplicated results
agent_with_expansion = build_agent(
    vector_store=vector_store,
    embedding_model=embedding_model,
    llm=llm,
    top_k=RETRIEVER_TOP_K,
    number_query_expansion=3,
)
answer_expanded = await ask(agent_with_expansion, QUERY_EXPANSION)

print(f"\nChunks retrieved without expansion : {len(results_no_exp)}")
print("(with expansion the agent internally deduplicates across all variant queries)")

---

## 4. Probing Failure Modes

The corpus was designed with three deliberate challenges. Run the queries below and observe the answers.

### 4a: Out-of-Portfolio Query

The **Lara Pallet** does not exist. A good RAG must say so instead of describing a different pallet.

In [ ]:
answer_ook = await ask(agent, "What materials is the Lara pallet made from?")

### 4b: Missing Data (LogyLight Pallet)

The LogyLight datasheet marks all LCA fields as *"not yet available"*. The correct answer is that we don't have the data, not a fabricated figure.

In [ ]:
answer_gap = await ask(agent, "What is the GWP of the LogyLight pallet?")

### 4c: Conflicting Evidence (Relicyc GWP Figures)

The 2021 Relicyc datasheet reports **4.1 kg CO₂e** per pallet. The 2023 EPD (third-party verified) reports a different, more recent figure. The RAG should flag the conflict and prefer the verified, more recent source.

In [ ]:
answer_conflict = await ask(
    agent, "What is the GWP of the Logypal 1 pallet, and how reliable is the figure?"
)

### 4d: Unverified Supplier Claim (Tesa ECO Tape)

The tesa supplier brochure claims **68% CO₂ reduction** compared to conventional tape. This is a self-declared marketing claim, there is no independent EPD. The RAG should report the claim but flag that it is unverified.

In [ ]:
answer_claim = await ask(
    agent,
    "How much lower is the carbon footprint of tesa ECO tape compared to standard tape?",
)

> 💬 **Discuss with your group:** Which of the four failure modes above concerns you most in a real deployment? What would the consequence be if a colleague trusted a wrong answer about a sustainability claim?
>
> Can you think of other ways the system might fail that aren't shown here?

---

## 5. Multi-Turn Conversation

The `ask()` function accepts a `history` argument, a list of prior `LLMMessage` objects. When history is provided the agent first **rewrites the query** to be self-contained (*"it"* becomes the actual product name) before retrieval.

This prevents the retriever from embedding vague pronouns that match nothing in the corpus.

In [ ]:
from conversational_toolkit.llms.base import LLMMessage, Roles

history: list[LLMMessage] = []


async def conversation_turn(query: str) -> str:
    global history
    answer = await agent.answer(QueryWithContext(query=query, history=history))
    history.append(LLMMessage(role=Roles.USER, content=query))
    history.append(LLMMessage(role=Roles.ASSISTANT, content=answer.content))
    return answer.content


# Turn 1: ask about a specific product
reply1 = await conversation_turn(
    "Which pallets in our portfolio have a third-party verified EPD?"
)
print("User: Which pallets in our portfolio have a third-party verified EPD?")
print(f"Assistant: {reply1}\n")

# Turn 2: follow-up using a pronoun — the agent should resolve "it" before retrieval
reply2 = await conversation_turn(
    "What is the GWP figure reported in it for the Logypal 1?"
)
print("User: What is the GWP figure reported in it for the Logypal 1?")
print(f"Assistant: {reply2}")

---

## 6. Running the Full Pipeline in One Call

The `run_pipeline()` convenience function executes all five steps end-to-end. It is also
what the `__main__` entry point calls.

Use it for quick one-shot queries. Use the individual step functions above when you need
to inspect intermediate results or iterate on a specific stage.

In [ ]:
from sme_kt_zh_collaboration_rag.feature0_baseline_rag import run_pipeline

answer = await run_pipeline(
    backend=BACKEND,
    query="What sustainability certifications do the pallets in the portfolio have?",
    reset_vs=False,
)
print(answer)

---

## 7. Switching LLM Backends

The pipeline abstracts the LLM behind a common interface. Only `build_llm()` needs to change.

| Backend | `BACKEND=` | Prerequisite |
|---|---|---|
| Ollama (local) | `"ollama"` | `ollama serve` + `ollama pull mistral-nemo:12b` |
| OpenAI | `"openai"` | `OPENAI_API_KEY` env variable |

You can also override the model within a backend:

```python
llm = build_llm(backend="openai", model_name="gpt-4o") # stronger model
llm = build_llm(backend="ollama", model_name="llama3.2") # smaller local model
```

The RAG pipeline is **backend-agnostic**, the retrieval step is identical regardless of which LLM is used.

In [ ]:
# Test openai

QUERY = "What materials is the Lara pallet made from?"

llm_openai = build_llm(backend="openai", model_name="gpt-4o-mini")
agent_openai = build_agent(vector_store, embedding_model, llm_openai)
answer_openai = await ask(agent_openai, QUERY)

---

## Tasks & Discussion

Work through these as a group. You don't need to do them all — pick what interests you or what matches your background. Mix and match freely.

---

### 💬 Explore & Discuss

**A. Build a test set together**
Go through the data files (run the scratch cell below to see what's there) and write down five questions where the answer *is* in the documents, two questions about products that don't exist or data that isn't there, and one question where you suspect conflicting information. Run them through the system. Does it pass your test?

**B. Evaluate the outputs — would you trust them?**
Look back at the failure mode answers in Section 4. For each one: would you have trusted this answer without knowing it might be wrong? What would a *good* response look like? What would need to change in the system to prevent this failure?

**C. Think about your own context**
If you were to deploy this in your organisation, what documents would go into the knowledge base? What questions should it answer well? Who would use it, and what are the consequences of a wrong answer?

---

### 🔧 Code Experiments

**1. Retrieval inspection**
Call `inspect_retrieval()` with several different queries. Look at the L2 scores — at roughly what value do retrieved chunks stop being relevant? Could you use this as a confidence threshold?

**2. Top-k sensitivity**
Change `top_k` from 5 to 1 and re-run a query from the failure mode section. Does the answer change? Try 10. Is more context always better, or does noise creep in?

**3. System prompt ablation**
In `feature0_baseline_rag.py`, find `SYSTEM_PROMPT`. Remove the rule about flagging conflicting information. Rebuild the agent and re-run the Relicyc GWP query (Section 4c). Does the answer change?

**4. Query phrasing experiment**
Try these three phrasings for the same underlying question: `"CO₂ footprint Logypal 1"`, `"carbon emissions recycled pallet"`, and `"GWP A1-A3 EPD pallet"`. Do the retrieved chunks differ? Which phrasing finds the most relevant result?

**5. Query expansion**
Set `number_query_expansion=3` (Step 5b) and repeat Task 4. Does expansion reduce sensitivity to phrasing?

---

### 🚀 Possible Extensions

| Extension | Effort | What it enables |
|---|---|---|
| Load the full corpus (`max_files=None`) | Minimal | More documents, better coverage |
| Try `gpt-4o` instead of `gpt-4o-mini` | Minimal | Stronger failure mode handling |
| Add a new document to `data/` | Easy | Extend the knowledge base with your own materials |
| Multi-lingual queries | Easy to test | Ask questions in German / French |
| Score threshold filter | Medium | Detect when a query falls outside the known portfolio |
| Structured outputs (Feature Track 3) | Medium | Every answer includes: product, figure, source, verified? |
| Query expansion + re-ranking (Feature Track 4) | Medium | Better recall for domain-specific vocabulary |
| Chat frontend (Feature Track 6) | Medium | Share the prototype with colleagues via a browser interface |

In [ ]:
# Scratch cell — run your experiments here
async def quick_retrieve(query: str, top_k: int = 5):
    retriever = VectorStoreRetriever(embedding_model, vector_store, top_k=top_k)
    results = await retriever.retrieve(query)
    print(f"Query: {query!r}  (top_k={top_k})")
    for r in results:
        src = r.metadata.get("source_file", "?")
        print(f"  score={r.score:.4f}  {src}  {r.title!r}")


await quick_retrieve("PFAS-free tape declaration")

---

## Summary

| Step | Function |
|---|---|
| 1. Load & chunk | `load_chunks(max_files)` |
| 2. Embed & index | `build_vector_store(chunks, emb, reset)` |
| 3. Inspect retrieval | `inspect_retrieval(query, vs, emb)` |
| 4. Build agent | `build_agent(vs, emb, llm, top_k, number_query_expansion)` |
| 5. Generate answer | `ask(agent, query, history)` |

### Three Core Failure Modes

| Failure mode | Symptom | Root cause | Addressed in |
|---|---|---|---|
| **Wrong entity** | RAG answers about the wrong product | Retriever returns high-scoring chunks for a product that doesn't exist in corpus; LLM picks the closest match | Feature Track 3 (structured output + confidence filtering) |
| **Missing data** | LLM invents a figure that isn't in any document | Retriever returns vaguely related chunks; LLM fills the gap with general knowledge | Feature Track 3 (strict source-only rules) |
| **Low recall** | Correct chunk exists but isn't retrieved | Query phrasing doesn't match document vocabulary; top-k too small | Feature Track 4 (query expansion, hybrid search) |

> **With OpenAI models these failures are often subtle** — GPT-4o follows the "say I don't know" rule reliably. They become more pronounced with local models (Ollama) or with adversarial query phrasing. The experiments in the Tasks section above give guidance on how to observe them.

**Next — Feature Track 1:** Explore chunking strategies and understand how chunk size and strategy affect retrieval quality.